In [ ]:
import import_ipynb
import chess

from Constants import *

importing Jupyter notebook from Constants.ipynb


In [ ]:
piece_values

{'p': 100, 'n': 320, 'b': 330, 'r': 500, 'q': 900, 'k': 20000}

### evaluate_move

This function calculates the current score of the board: the score of all pieces is taken into account and summed up.

##### Member of class
    chess.Board

##### Arguments
    N/A

##### Returns
    Evaluated score of the chess board.

##### Side effects
    N/A


In [ ]:
def evaluate_move(self, color: chess.Color, is_opponent: bool, prev_score: int, move: chess.Move) -> int:
    score_diff = 0
    is_opponent_multiplier = -1 if is_opponent else 1

    # If the move is an attack, add score based on the opponent's lost piece
    to_piece = self.piece_at(move.to_square)
    if to_piece is not None: # No need to check if the piece is of the opposite color - attacking one's own color is not a legal move
        score_diff += piece_values[chess.piece_symbol(to_piece.piece_type)]

    # Consider positional score
    from_piece = self.piece_at(move.from_square)
    from_piece_symbol = chess.piece_symbol(from_piece.piece_type).lower()
    from_piece_square_table_key = ('ke' if self.ending else 'km') \
        if from_piece_symbol == 'k' \
        else from_piece_symbol
    score_diff -= self.get_positional_score(color, from_piece_square_table_key, move.from_square)
    score_diff += self.get_positional_score(color, from_piece_square_table_key, move.to_square)        

    # Handle rook during castling
    if self.is_castling(move):
        old_square, new_square = self.find_castling_rook_squares(color, move)
        score_diff -= self.get_positional_score(color, 'r', old_square)
        score_diff += self.get_positional_score(color, 'r', new_square)

    return prev_score + (score_diff * is_opponent_multiplier)

chess.Board.evaluate_move = evaluate_move

In [ ]:
def get_positional_score(self, color: chess.Color, piece_square_table_key: str, square: chess.Square) -> int:
    hor_index = square % 8
    ver_index = 7 - square // 8 if color == chess.WHITE else square // 8
    piece_square_value = piece_square_tables[piece_square_table_key][ver_index][hor_index]
    return piece_square_value

chess.Board.get_positional_score = get_positional_score

In [ ]:
def find_castling_rook_squares(self, color: chess.Color, move: chess.Move) -> (chess.Square, chess.Square):
    king_move_dir = 1 if move.to_square > move.from_square else -1
    old_rook_square = (chess.A1 if king_move_dir == -1 else chess.H1) \
        if color == chess.WHITE \
        else (chess.A8 if king_move_dir == -1 else chess.H8)
    new_rook_square = move.to_square - king_move_dir

    return old_rook_square, new_rook_square

chess.Board.find_castling_rook_squares = find_castling_rook_squares

--- 
<br>

## get_move_score

This function calculates the evaluation score of a given move.

#### Member of class:
    chess.Board

#### Arguments
    move: chess.Move
        The move to be evaluated.

#### Returns
    The evaluated score of the move.

#### Side effects
    - If the function is interrupted, the board may be in a different state than before it was called.


In [ ]:
def get_move_score(self, move: chess.Move) -> int:
    self.push(move)
    move_score = self.get_ai_score()
    self.pop()
    
    return move_score

chess.Board.get_move_score = get_move_score

### get_state_string

Returns a string that represents the current state of a chess board, i.e. which pieces are at which position.

##### Member of class
    chess.Board

##### Arguments
    N/A

##### Returns
    A string that represents the state of the board.

##### Side effects
    N/A


In [ ]:
def get_state_string(self) -> str:
    return ''.join([
        '-' if self.piece_at(square) is None else self.piece_at(square).symbol()
            for square in chess.SQUARES
    ])

chess.Board.get_state_string = get_state_string

In [ ]:
def check_and_set_ending(self) -> None:
    if self.ending: return

    meets_ending_requirement = { chess.WHITE: False, chess.BLACK: False }
    for color in [ chess.WHITE, chess.BLACK ]:
        # Option 1: no queen
        if self.piece_count(color, chess.QUEEN) == 0:
            meets_ending_requirement[color] = True
            continue

        # Option 2: at most one minor piece and no other pieces (besides queen)
        # Most sources state that pawns aren't pieces, so these are not counted
        if self.piece_count(color, chess.BISHOP, chess.KNIGHT) <= 1 and self.piece_count(color, chess.ROOK) == 0:
            meets_ending_requirement[color] = True

    if meets_ending_requirement[chess.WHITE] and meets_ending_requirement[chess.BLACK]:
        self.ending = True

chess.Board.check_and_set_ending = check_and_set_ending

In [ ]:
def piece_count(self, color: chess.Color, *piece_types: chess.PieceType) -> int:
    return sum( len(self.pieces(piece_type, color)) for piece_type in piece_types )

chess.Board.piece_count = piece_count

### get_search_result_if_finished

Checks if a search (minimax or alpha-beta) is finished. This is the case if either the game is over with the board state at the current search node, or if the maximum depth has been reached. If the search is finished, it returns the result of this search.

##### Member of class
    chess.Board

##### Arguments
    iteration: int
        The current depth of the search.
    max_iterations: int
        The maximum depth of the search.

##### Returns
    If the search is finished:
        tuple(best_score, best_move)

        best_score: int
            The board score after making the recommended best move.
        best_move: chess.Move
            The recommended best move to make.
            
    Otherwise:
        None

##### Side effects
    N/A

In [ ]:
def get_search_result_if_finished(self, iteration: int, max_iterations: int, eval_score: int) -> (int, chess.Move):
    # If the game has finished, return a positive score if we won, or a negative score if we lost.
    # This score is higher than any possible score in the evaluation function, because wining or losing
    # the game is more impactful than any other board state.
    # If the game ended in a draw, return a neutral score (0).
    outcome = self.outcome()
    if outcome is not None:
        if outcome.winner == chess.WHITE:
            return 1000000, None
        if outcome.winner == chess.BLACK:
            return -1000000, None
        return 0, None # Draw

    # If the search has hit its max depth (max_iterations), return the current evaluation score for the AI
    if iteration >= max_iterations:
        return eval_score, None

    # The search has not ended, so no result
    return None

chess.Board.get_search_result_if_finished = get_search_result_if_finished

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d6ce9acd-52c5-4422-904d-8424da19408b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>